In [2]:
! pip install -q python-dotenv
! pip install -q neo4j
! pip install -q langchain
! pip install -q langchain-openai
! pip install -q tiktoken


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import dotenv
import os

dotenv.load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("URL")
os.environ["NEO4J_USERNAME"]= os.getenv("USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("PASSWORD2")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAIKEY")

In [4]:
from langchain_community.graphs import Neo4jGraph
import sys
sys.path.append('../utils')  
from helper import write_chunks_to_df


In [67]:
graphDB = Neo4jGraph()

In [6]:
#graphDB.query("SHOW INDEXES")

In [68]:
#
# Initialize Semantic Vector Index
#
# DOCS: https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/vector-indexes/
#

vector_index_query=""" 
CALL db.index.vector.createNodeIndex(
  'accreditation_index',
  'Chunk',
  'embedding',
   1536,
  'cosine'
)
"""

graphDB.query(vector_index_query)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.vector.createNodeIndex`: Caused by: org.neo4j.kernel.api.exceptions.schema.EquivalentSchemaRuleAlreadyExistsException: An equivalent index already exists, 'Index( id=5, name='accreditation_index', type='VECTOR', schema=(:Chunk {embedding}), indexProvider='vector-1.0' )'.}

In [69]:
graphDB.query("SHOW INDEXES")

[{'id': 5,
  'name': 'accreditation_index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 1,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 21, 6, 50, 2, 905000000, tzinfo=<UTC>),
  'readCount': 7370},
 {'id': 2,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 21, 6, 41, 9, 866000000, tzinfo=<UTC>),
  'readC

### Generate Vector Embeddings

In [70]:
#
#  Query All Chunks without embedding
#

all_chunks_query = """ 
MATCH (c:Chunk) 
WHERE c.embedding IS null OR c.embedding = 0
RETURN c
"""


# attach embedding to chunk.embedding
vector_to_chunk_query = """ 
MATCH (c:Chunk {UUID: $UUID})
SET c.embedding = $vector
"""

In [71]:
result = graphDB.query(all_chunks_query)

In [72]:
#
# Generate Vectors for Chunk.text and update Chunk.embedding
#

chunk_dataframe = write_chunks_to_df(result)

In [73]:
#chunk_dataframe.head()
chunk_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      248 non-null    object
 1   nodeType  248 non-null    object
 2   UUID      248 non-null    object
dtypes: object(3)
memory usage: 5.9+ KB


In [74]:
#
# Chunk embedding function
# 
# DOCS: https://platform.openai.com/docs/guides/embeddings/use-cases
#


from openai import OpenAI

client =OpenAI()

#MODEL =  "text-embedding-3-small"
MODEL = "text-embedding-ada-002"

def get_embedding(text, model = MODEL):

    text = text.replace("\n"," ")

    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [75]:
chunk_dataframe['vector'] = chunk_dataframe['text'].apply(lambda x:get_embedding(x))

In [76]:
chunk_dataframe.head()

,text,nodeType,UUID,vector
0,programs student learning outcomes evidence le...,DATACHUNK,a4f1f358-6e65-4eea-a6a9-b6ed5793567b,"[0.0025794648099690676, -0.0022454087156802416..."
1,accredited unit recently completed regular aca...,DATACHUNK,3cc540e9-1cd9-4bb2-a268-598610a0be5a,"[0.004533860366791487, 0.0007574598421342671, ..."
2,research service contribute mission school . h...,DATACHUNK,c6bea969-903c-40c3-ad90-39cdf51dc4b2,"[0.0023216428235173225, 0.0006858606939204037,..."
3,consistent school mission strategic initiative...,DATACHUNK,1c66a35a-a2b8-4bb4-a32e-ba3d19b3f7fd,"[-0.004242770839482546, 0.0069182999432086945,..."
4,programs locations modalities take considerati...,DATACHUNK,5777adc9-c5f2-46bb-b4ef-0cf3fca4839e,"[0.0066562071442604065, 0.0010074122110381722,..."


In [77]:
for i, row in chunk_dataframe.iterrows():

    graphDB.query(vector_to_chunk_query, 
                params ={
                    'UUID':row['UUID'],
                    'vector':row['vector']
                })

### Queries

In [78]:
semantic_index_query = """

CALL db.index.vector.queryNodes('accreditation_index', 2, $inputVector)
YIELD node AS responseNode, score

RETURN responseNode.text, score 
"""

In [79]:
query_text = "what are aacsb standards"
query_vector = get_embedding(query_text)
query_result = graphDB.query(semantic_index_query, 
                                params={
                                    "inputVector":query_vector
                                })

In [80]:
query_result

[{'responseNode.text': '3.1 faculty sufficiency school adopts applies criteria documenting faculty members participating supporting consistent mission . school adapt guidance particular situation mission developing implementing criteria indicate school meeting spirit intent standard . criteria address activities required attain participating supporting status depth breadth activities expected within typical aacsb accreditation review cycle maintain participating supporting status . criteria periodically reviewed',
  'score': 0.908052921295166},
 {'responseNode.text': 'attain participating supporting status depth breadth activities expected within typical aacsb accreditation review cycle maintain participating supporting status . criteria periodically reviewed reflect focus continuous improvement . normally participating faculty members deliver least 75 percent school teaching globally i.e . across entire accredited unit participating faculty members deliver least 60 percent teaching wi

In [10]:
#
# Standard QA
#
from langchain_community.vectorstores import Neo4jVector

VECTOR_INDEX_NAME = 'accreditiation-index'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY='embedding'


In [ ]:
#https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever/


In [33]:
import logging
from datetime import datetime
import json

log_dir = os.path.abspath('../logs')
os.makedirs(log_dir, exist_ok=True)

#Info logger
info_logger = logging.getLogger('info_logger')
info_logger.setLevel(logging.INFO)
info_handler = logging.FileHandler(os.path.join(log_dir, 'info.log'))
info_logger.addHandler(info_handler)

#Error logger
error_logger = logging.getLogger('error_logger')
error_logger.setLevel(logging.ERROR)
error_handler = logging.FileHandler(os.path.join(log_dir, 'error.log'))
error_logger.addHandler(error_handler)



In [40]:
# sucessful test
#info_logger.info("testing info logger")
#error_logger.error("testing error logger")

In [12]:

#
# Sourced from documentation and modified: https://python.langchain.com/docs/integrations/graphs/neo4j_cypher/
#

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Perfect Syntax: Your queries must be in the correct Cypher syntax, at all costs you should avoide
'CypherSyntaxError' and 'ValueError' resulting from your query ie:
ValueError: Generated Cypher Statement is not valid
code: Neo.ClientError.Statement.SyntaxError message: Invalid input 'objective': expected ")", "WHERE", or a parameter (line 1, column 19 (offset: 18))
"MATCH (n:Learning objective) RETURN n.name"

Examples: Here are a few examples of generated Cypher statements for particular questions:

## AACSB STANDARD EXAMPLE
# Which standards deal with staff resources?
MATCH (n)
WHERE n.nodeCat = 'AACSB' AND (n.text CONTAINS 'staff' AND n.text CONTAINS 'resources' OR n.text CONTAINS 'staff resources')
RETURN n

# How is standard 2 documented
MATCH (d:Documentation) WHERE d.parentStandardNum = 2 RETURN d.text

## INSTITUTION EXAMPLE -- schema may vary from example, reference schema
# Which learning objectives did undergradute and graduate program evaluate
MATCH (p:Program)-[]->(l:`Learning objective`)
WHERE (p.name CONTAINS 'undergraduate' OR p.name CONTAINS 'graduate') 
RETURN l.name






The question is:
{question}"""

In [14]:
#
# RExample of returning nodes:
# MATCH (p:Program)-[]->(l:`Learning objective`)
# WHERE (p.name CONTAINS 'undergraduate' OR p.name CONTAINS 'graduate') 
# RETURN l,p
#

In [13]:
graphDB.refresh_schema()
schema = graphDB.schema


In [18]:
import pprint

pp = pprint.PrettyPrinter(indent = 2)
pp.pprint(schema)

('Node properties are the following:\n'
 'Root {nodeCat: STRING, Title: STRING, nodeType: STRING},Standard {title: '
 'STRING, nodeType: STRING, standardNum: INTEGER, parentSectionNum: INTEGER, '
 'text: STRING, source: STRING, year: INTEGER, nodeCat: STRING},Chunk '
 '{parentType: STRING, standardTitle: STRING, chunkSeqNum: INTEGER, embedding: '
 'INTEGER, parentId: INTEGER, parentStandardNum: INTEGER, UUID: STRING, '
 'nodeCat: STRING, nodeType: STRING, text: STRING, title: STRING, sectionNum: '
 'INTEGER, parentDocId: STRING, source: STRING, inputIdx: INTEGER, '
 'sourcePageIdxNum: INTEGER},Basis {nodeCat: STRING, nodeType: STRING, text: '
 'STRING, parentStandardNum: INTEGER, standardTitle: STRING},Documentation '
 '{nodeType: STRING, text: STRING, parentStandardNum: INTEGER, standardTitle: '
 'STRING, nodeCat: STRING},Definitions {nodeType: STRING, text: STRING, '
 'parentStandardNum: INTEGER, standardTitle: STRING, nodeCat: STRING},Section '
 '{title: STRING, nodeType: STRING, so

In [34]:
# Query the knowledge graph in a RAG application
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate



graphDB.refresh_schema()

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)


cypher_chain = GraphCypherQAChain.from_llm(
    graph=graphDB,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    validate_cypher=True, # Validate relationship directions
    verbose= True,
    return_intermediate_steps=True
)

In [33]:
# #cypher_chain.invoke({"query": "How many learning objectives are assessed"})
# #result = cypher_chain.invoke({"query": "Which student learning goals were identified"}) # does not know
# result = cypher_chain.invoke({"query": "What is standard 5 about"}) # does not know #works well

# #cypher_chain.invoke({"query": "What are the descriptions of the Learning goal"}) ## does not know , see still Learning goal example
# #cypher_chain.invoke({"query": "Tell me about AACSB standards"}) ## does not know , see still Learning goal example
# print(result['result'])
# print(result['intermediate_steps'])





> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Standard {standardNum: 5}) RETURN s.title, s.text
Full Context:
[{'s.title': 'Assurance of Learning', 's.text': 'standard 5 assurance learning 5.1 school uses well documented assurance learning aol processes include direct indirect measures ensuring quality degree programs deemed scope accreditation purposes . results school aol work leads curricular process improvements . 5.2 programs resulting degree credential structured designed ensure equivalence high quality outcomes irrespective location modality instructional delivery . 5.3 microlearning credentials stackable otherwise able combined aacsb accredited degree program include processes ensure high quality continuous improvement . 5.4 non degree executive education generates greater five percent school total annual resources include processes ensure high quality continuous improvement .'}]

> Finished chain.
Standard 5 is about assurance of learning, which inclu

Out of the box LLM genreate cypher queries are not producing any results, OR are producing errors.Provide some examples to the mode an oppopriate queries based on the schema. 

In [34]:
# from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
# chain  = RetrievalQAWithSourcesChain.from_chain_type(

# )

In [54]:
#
#  LangChain Docs: https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever/
#     



In [35]:

import os
import re
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)


def extract_list(llm_result):
    content = llm_result.content
    return content.split("\\n")

def generate_multi_question(question):

    system_prompt = f""" 

    # Instruction
    You are an AI language model assistant. Your task is to generate three 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by new line. 
    
    # Format Rules
    DO NOT NUMBER THE LIST
    Original question: {question}
    """ 
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
    result = llm.invoke(system_prompt)
    query_list = extract_list(result)
     



    return {'original_query': question, 'generated_query_list': query_list}

In [36]:
from typing import List, Dict





def generate_multi_question_aacsb(question: str) -> Dict[str, any]:
    """
    For queries relating to the AACSB standards, generates list three different versions
    of the user's input query. Used downstream for multiquery retrieval.

    Args:
        question (str): The user's input query.

    Returns:
        Dict[str, any]: A dictionary containing the original user question and the generated list of alternative 
        questions. Keys include 'original_query' for the original question and 'generated_query_list' for the list 
        of alternative questions.

    """


    system_prompt = f""" 

    # Instruction
    You are an AI language model assistant. Your task is to generate three 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search, by providing precise questions.
    Provide these alternative questions separated by new line. 

    # Important Context: AACSB
    The questions you generate are directly related to extracting useful information on the 
    AACSB accrediation standards. The questions you generate will be used as vector database index
    queries that contain information on:

    - formal AACSB descriptions
    - documentation that supports each standard 
    - basis for evaluation of the standards 
    - relevent definitions of terms used in the standard descriptions.

    The AACSB Website provides the following summary of their work:
    AACSB accreditation is known, worldwide, as the longest-standing, most recognized form of 
    specialized accreditation that an institution and its business programs can earn. 
    Accreditation is a voluntary, nongovernmental process that includes a rigorous external review 
    of a school's mission, faculty qualifications, curricula, and ability to provide the highest-quality programs.
    
    # Format Rules
    DO NOT NUMBER THE LIST
    DO NOT ANSWER THE QUESTION
    Original question: {question}
    """ 
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
    result = llm.invoke(system_prompt)
    query_list = extract_list(result)  

    log_data = {
        'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
        'level': 'INFO',
        'function_name': 'generate_multi_question_institution',
        'original_query': question,
        'generated_query_list': query_list
    }
    info_logger.info(json.dumps(log_data))

    return {'original_query': question, 'generated_query_list': query_list, 'cat':'AACSB'}

In [37]:
from typing import List, Dict


def generate_multi_question_institution(question: str) -> Dict[str, any]:
    """
    For school specific, or academic institution specific, queries. Generates list three different versions
    of the user's input query. Used downstream for multiquery retrieval.

    Args:
        question (str): The user's input query.

    Returns:
        Dict[str, any]: A dictionary containing the original user question and the generated list of alternative 
        questions. Keys include 'original_query' for the original question and 'generated_query_list' for the list 
        of alternative questions.
    """


    system_prompt = f""" 

    # Instruction
    You are an AI language model assistant. Your task is to generate three 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search, by providing precise questions.
    Provide these alternative questions separated by new line. 

    # Important Context: Academic Institution
    The questions you generate are directly related to extracting useful information about a School
    of Business.  The questions you generate will be used as vector database index
    queries that contain information on:

    - Strategic Plan, Mission and Fiscal Resources
    - Academic Departments in the School of Business inluding not limited to : Accounting, Marketing, Management, Finance, Entreprenuership
    - Student Services and Student Organizations 
    - Program Goals, Learning Objectives and Curriculum Assessment
    - Continuous Improvement


    # Format Rules
    DO NOT NUMBER THE LIST
    DO NOT ANSWER THE QUESTION
    Original question: {question}
    """ 
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
    result = llm.invoke(system_prompt)
    query_list = extract_list(result)  

    log_data = {
    'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
    'level': 'INFO',
    'function_name': 'generate_multi_question_institution',
    'original_query': question,
    'generated_query_list': query_list
        }
    info_logger.info(json.dumps(log_data))                                                                      

    return {'original_query': question, 'generated_query_list': query_list, 'cat':'INSTITUTION'}

In [38]:
from typing import List, Dict


def generate_sub_questions_hybrid(question: str) -> Dict[str, any]:
    """
    For queries relating to both AACSB accreditation AND Academic Institution (School) information .
    Generates two distinct sub questions based on the user's input query. Used downstream for multiquery retrieval.

    Args:
        question (str): The user's input query.

    Returns:
        Dict[str, any]: A dictionary containing the original user question and the generated list of alternative 
        questions. Keys include 'original_query' for the original question and 'generated_query_list' for the list 
        of alternative questions.
    """


    system_prompt = f""" 

    # Instruction
    You are an AI language model assistant. Your task is to evalute the users input query and 
    divide the query into two and ONLY TWO sub questions. The first sub questions should address the portion
    of the user query that relates to the AACSB Standards, the second subquestion should relate the institution
    specific portion of the query. Your overall objective is to break down the complex user query into the 
    two distinct sub questions.  Provide these alternative questions separated by new line. 

    # Important Context: Sub 
    
    ## 1. AACSB Standard sub question:
    AACSB sub question may related to accreditation content such as:

    - formal AACSB descriptions
    - documentation that supports each standard 
    - basis for evaluation of the standards 
    - relevent definitions of terms used in the standard descriptions.

    The AACSB Website provides the following summary of their work:
    AACSB accreditation is known, worldwide, as the longest-standing, most recognized form of 
    specialized accreditation that an institution and its business programs can earn. 
    Accreditation is a voluntary, nongovernmental process that includes a rigorous external review 
    of a school's mission, faculty qualifications, curricula, and ability to provide the highest-quality programs.


    ## 2. Academic Instiution sub question:
    Academic Instiution sub question may relate to School of Business content such as:

    - Strategic Plan, Mission and Fiscal Resources
    - Academic Departments in the School of Business inluding not limited to : Accounting, Marketing, Management, Finance, Entreprenuership
    - Student Services and Student Organizations 
    - Program Goals, Learning Objectives and Curriculum Assessment
    - Continuous Improvement


    # Format Rules
    DO NOT NUMBER THE LIST
    DO NOT ANSWER THE QUESTION
    Original question: {question}
    """ 
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
    result = llm.invoke(system_prompt)
    query_list = extract_list(result)  

    log_data = {
        'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
        'level': 'INFO',
        'function_name': 'generate_multi_question_institution',
        'original_query': question,
        'generated_query_list': query_list
    }
    info_logger.info(json.dumps(log_data))

    return {'original_query': question, 'generated_query_list': query_list, 'cat':'HYBRID'}

In [83]:
# test_query = "how should I prepare for extended travel"

# result = generate_multi_question(test_query)

In [84]:
# for r in result['generated_query_list']:
#     print (r)

- What are some tips for preparing for an extended travel?
- Can you provide advice on getting ready for extended travel?
- What steps should I take to prepare for a long trip?
- How can I best get myself ready for an extended travel?
- Are there any recommendations for preparing for a lengthy journey?


In [93]:

# test_query = "Our accountind department recently  updated the curriculum to include carbon footprint, does this reflect the sustainability standard "
# test_hybrid_query = generate_sub_questions_hybrid(test_query)

In [94]:
# test_hybrid_query

{'original_query': 'Our accountind department recently  updated the curriculum to include carbon footprint, does this reflect the sustainability standard ',
 'generated_query_list': ['Alternative questions:\n1. What are the formal AACSB descriptions and documentation that support the sustainability standard?\n2. How does the updated curriculum in our accounting department align with the sustainability standard?'],
 'cat': 'HYBRID'}

In [39]:
from openai import OpenAI
import json

client = OpenAI()



def optimize_query_function_route(user_query:str)->str:
    """
    Run a conversation with OpenAI's language model, providing the user query and available functions to the model.

    Args:
        user_query (str): The user's input query.

    Returns:
        str: The response from the language model.

    Step 1: Send the conversation and available functions to the model.
        - Each function is described with its name, description, and parameters.
        - Three functions are available:
            1. generate_multi_question_aacsb: Generates three different versions of the user's input query
               related to AACSB standards.
            2. generate_multi_question_institution: Generates three different versions of the user's input query
               for school-specific or academic institution-specific queries.
            3. generate_sub_questions_hybrid: Generates two distinct sub-questions based on the user's input query,
               relating to both AACSB accreditation and academic institution (school) information.

    Returns the response from the language model, specifically the first choice message from the available choices.
    """


    messages = [{"role": "user", "content": user_query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "generate_multi_question_aacsb",
                "description": "For queries relating to the AACSB standards, generates list three different versions of the user's input query. Used downstream for multiquery retrieval.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "Input user query",
                        }
                    },
                    "required": ["question"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "generate_multi_question_institution",
                "description": "For school specific, or academic institution specific, queries. Generates list three different versions of the user's input query. Used downstream for multiquery retrieval.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "Input user query",
                        }
                    },
                    "required": ["question"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "generate_sub_questions_hybrid",
                "description": "For queries relating to both AACSB accreditation AND Academic Institution (School) information. Generates two distinct sub questions based on the user's input query. Used downstream for multiquery retrieval.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "Input user query",
                        }
                    },
                    "required": ["question"],
                },
            },
        }

    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default
    )
    return response.choices[0].message
    


In [102]:
# input_query = "AACSB standards on sustainability"
# result = optimize_query_function_route(input_query)

In [108]:
# import json

# function_name =  result.tool_calls[0].function.name
# function_args = json.loads(result.tool_calls[0].function.arguments)['question']
# print(f"{function_name} , {function_args}, ")

generate_multi_question_aacsb , AACSB standards on sustainability, 


In [109]:
# function = globals()[function_name]
# print(function(function_args))

{'original_query': 'AACSB standards on sustainability', 'generated_query_list': ['1. What are the AACSB accrediation standards on sustainability?\n2. How does AACSB define sustainability in their accrediation standards?\n3. Can you provide information on the AACSB standards related to sustainability?']}


In [40]:
import json

def execute_route_function(conv_result: any)->any:
    """
    Execute a route function based on the provided conversation result.

    Args:
        conv_result (any): The conversation result containing information about the function to execute.

    Returns:
        any: The result of executing the route function.

    Extracts the name and arguments of the route function from the conversation result and then
    dynamically executes the function using the extracted information.

    Note:
        - The function_name and function_args are extracted from the tool_calls attribute of the conversation result.
        - The function_name is used to retrieve the actual function from the global namespace.
        - The function_args are passed as arguments to the retrieved function.
    """

    function_name =  conv_result.tool_calls[0].function.name
    function_args = json.loads(conv_result.tool_calls[0].function.arguments)['question']
    function = globals()[function_name]
    return function(function_args)





In [41]:
def query_routing_pipeline(user_query):
    """
    Run a conversation with OpenAI's language model using the provided user query and then execute the route function.

    Args:
        user_query (str): The user's input query.

    Returns:
        dict: The result of executing the route function.

    This function serves as a pipeline for querying and routing based on the user's input.
    It first runs a conversation with OpenAI's language model using the provided user query.
    The result of the conversation is then passed to the execute_route_function to determine and execute the appropriate route function.
    The result of executing the route function is returned as a dictionary.
    """
    result = optimize_query_function_route(user_query)
    return execute_route_function(result) #dict



In [126]:

# test_query_in = "AACSB standards on sustainability"
# pipeline_result = query_routing_pipeline(test_query_in)

In [127]:
# pipeline_result

{'original_query': 'AACSB standards on sustainability',
 'generated_query_list': ['1. What are the AACSB standards for sustainability?\n2. Can you provide information on the AACSB accreditation standards related to sustainability?\n3. How does the AACSB incorporate sustainability into its accreditation standards?'],
 'cat': 'AACSB'}

In [60]:

q = {'original_query': 'AACSB standards on sustainability',
 'generated_query_list': ['1. What are the AACSB standards for sustainability?\n2. Can you provide information on the AACSB accreditation standards related to sustainability?\n3. How does the AACSB incorporate sustainability into its accreditation standards?'],
 'cat': 'AACSB'}

query_vector = get_embedding(q['generated_query_list'][0])
query_result = graphDB.query(semantic_index_query, 
                    params={
                        "inputVector":query_vector
                    })
query_text = [text['responseNode.text'] for text in query_result]

In [81]:
print(query_vector)
print(query_result)
print(query_text)

[-0.004673096351325512, 0.005768295843154192, 0.013911989517509937, -0.08098553866147995, -0.028134780004620552, 0.022081583738327026, -0.04780396446585655, 0.0003510372480377555, 0.01629478856921196, -0.007836594246327877, 0.018781187012791634, 0.01941758580505848, -0.012757590971887112, 0.01255039032548666, 0.019831985235214233, -0.002910047769546509, 0.023102782666683197, -0.0034483973868191242, 0.0032263975590467453, -0.014555789530277252, -0.010826191864907742, 0.002062748419120908, -0.02091238461434841, 0.008117794059216976, 0.004510296508669853, 0.011033391579985619, 0.02461238205432892, -0.020808784291148186, -0.015969188883900642, -0.009605192579329014, 0.003966397140175104, -0.009087193757295609, -0.011321991682052612, 0.004983896389603615, -0.007233494892716408, -0.00413659680634737, -0.02304358221590519, -0.02536718174815178, 0.03179037570953369, 0.011277591809630394, -0.01190659124404192, 0.006900494918227196, -0.02436078153550625, 0.0011497741797938943, 0.0047100964002311

In [83]:
def retriever(query_dict, cypher_chain=cypher_chain):

    
    context_data = []

    if query_dict['cat'] != 'HYBRID':
        # multiqueries -- case A
        # single cypher query for original query
        for q in query_dict['generated_query_list']:
            query_vector = get_embedding(q)
            query_result = graphDB.query(semantic_index_query, 
                                params={
                                    "inputVector":query_vector
                                })
            query_text = [text['responseNode.text'] for text in query_result]

            log_data = {
            'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
            'level': 'INFO',
            'function_name': 'retriever()',
            'original_query': query_dict['original_query'],
            'semantic_query_result': query_text,

                }
            info_logger.info(json.dumps(log_data))



            context_data.extend(query_text)
        
        #cypher_result = cypher_chain.invoke({"query": query_dict['original_query']})

        try:
            cypher_result = cypher_chain.invoke({"query": query_dict['original_query']})
      
            log_data = {
                        'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
                        'level': 'INFO',
                        'function_name': 'retriever()',
                        'original_query': query_dict['original_query'],
                        'cypher_query': cypher_result['result'],
                        'cypher_steps': cypher_result['intermediate_steps']

                            }
            info_logger.info(json.dumps(log_data))
                
      
      
      
      
        except Exception as e:
                log_data = {
                        'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
                        'level': 'ERROR',
                        'function_name': 'retriever()',
                        'original_query': query_dict['original_query'],
                        'cypher_query_1': cypher_result,
                        'ErrorException': e
                        }
                error_logger.error(json.dumps(log_data))
                cypher_result = "" #set empty for context string down steam

        if "I don't know the answer" not in cypher_result['result']:
            context_data.append(cypher_result['result'])
            

    
    else:
        # subqueries -- case B
        # cypher query for each subquery
        for q in query_dict['generated_query_list']:
            query_vector = get_embedding(q)
            query_result = graphDB.query(semantic_index_query, 
                                params={
                                    "inputVector":query_vector
                                })
            query_text = [text['responseNode.text'] for text in query_result]

            log_data = {
            'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
            'level': 'INFO',
            'function_name': 'retriever()',
            'original_query': query_dict['original_query'],
            'semantic_query_result': query_text,

            }
            info_logger.info(json.dumps(log_data))
            context_data.extend(query_text)

            try:
                cypher_result = cypher_chain.invoke({"query": q})
                log_data = {
                        'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
                        'level': 'INFO',
                        'function_name': 'retriever()',
                        'original_query': query_dict['original_query'],
                        'cypher_query': cypher_result['result'],
                        'cypher_steps': cypher_result['intermediate_steps']

                            }
                info_logger.info(json.dumps(log_data))
                
            except Exception as e:
                log_data = {
                        'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
                        'level': 'ERROR',
                        'function_name': 'retriever()',
                        'original_query': query_dict['original_query'],
                        'cypher_query_2': cypher_result
                            }
                error_logger.error(json.dumps(log_data))
                cypher_result = "" #set empty for context string down steam

            if "I don't know the answer" not in cypher_result['result']:
                context_data.append(cypher_result['result'])
                
    # Final Context Str
    context_str = ', '.join(str(item) for item in context_data)

    # Log context_str 
  
    log_data = {
    'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
    'level': 'INFO',
    'function_name': 'retriever()',
    'original_query': query_dict['original_query'],
    'context_string': context_str
    }

    info_logger.info(json.dumps(log_data))



    #if not hybrid
    ### loop through multiqueries
    #### call embedding function on each and retrieve data from db
    #### append to  context data
    ### call cypher query, append results to cyher data

    #else:
    ## loop through sub queries
    ### call embedding and retrieve append resutls to context data
    ### call cypher, append results to context data

    return context_str #return context data as a string


In [43]:
#ret_result = retriever(pipeline_result)

In [45]:
# ret_result

In [52]:
def generator(original_query, context_string, model = "gpt-3.5-turbo-16k"):

    system_prompt = f"""
    # Instruction:
    You are an AI assistant generating a thorough and thoughtful response to a user query.
    You are to generate a response that answers their query solely based on the context provided below
    do not use any other outside information. The grounding context information has been retrieved from
    a database which is the absolute knowledge source. 

    # User Query:
    {original_query}

    # Grounding Context:
    {context_string}

    # Output Style
    Your tone should be professional. And your response should be detailed, as this information
    will be used to generate reports. 

    """

    if not context_string:
        log_data = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
                    'level': 'ERROR',
                    'function_name': 'generator()',
                    'original_query': original_query,
                    'context_string': 0

                        }
        error_logger.error(json.dumps(log_data))
        return "Generator Issue: context string is empty, unable to generate response, please rephrase your question and try again..."

    llm = ChatOpenAI(model=model, temperature=0)
    result = llm.invoke(system_prompt)
    content = result.content

    # Log Response
    log_data = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
                    'level': 'INFO',
                    'function_name': 'generator()',
                    'original_query': original_query,
                    'context_string': context_string,
                    'generatedResponse': content

                        }
    info_logger.info(json.dumps(log_data))


    return content

In [47]:
test_query = [
    "AACSB Standards on sustainability", #aacsb standards
    "Our accounting department updated curriculum to include environmental impact in business risk evaluation, does this reflect the sustanability standard", #hybrid subquestion()
    "Standard 5 specifies a systematic process for assurance of learning. What do peer review teams usually expect in determining whether this standard is met?",
    "What are intellectual contributions",
    "Must faculty members publish in order to be qualified as scholarly academic, practice academic, or scholarly practitioner?", #FAQ
    "Who is the dean of my school",
    "What are the management courses",
    "Which leaning objectives are assessed in the undergradute business program ",
    "How is standard 8 defined?",
    "What are MACC program students expected to demonstrate"
]


In [146]:
#gen = generator(test_query_in, ret_result)

In [53]:
def advRagPipeline(query):
    try:
        query_pipeline_result = query_routing_pipeline(query)
        context_str = retriever(query_pipeline_result)
        output = generator(query,context_str)

    except Exception as e:
        raise e

    return output


In [84]:
test_out = []


for q in test_query:
    output = advRagPipeline(q)
    test_out.append(output)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Standard) WHERE n.title = 'AACSB Standards on sustainability' RETURN n
Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Standard {standardNum: 5})<-[:BASIS_OF]-(b:Basis)
WHERE b.text CONTAINS 'systematic process for assurance of learning'
RETURN b.text, s.title
Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Co

In [86]:
for i, (q,o) in enumerate(zip(test_query,test_out)):
    print("======================================================")
    print("======================================================")
    print(f" {i}: {q} \n")
    print(f" {i}: {o} \n")
    print("======================================================")
    print("======================================================")



 0: AACSB Standards on sustainability 

 0: The AACSB (Association to Advance Collegiate Schools of Business) has established standards on sustainability that schools should adhere to. One of these standards is 3.1, which focuses on faculty sufficiency. According to this standard, schools are required to adopt and apply criteria that document faculty members' participation and support in line with the school's consistent mission.

The criteria should be adaptable to the specific situation of the school and should guide the development and implementation of activities that indicate the school is meeting the spirit and intent of the standard. These criteria should address the activities required to attain and maintain participating and supporting status within the typical AACSB accreditation review cycle.

It is important to note that these criteria are periodically reviewed, and the accreditation process includes a complete assessment of the degree program. Table 5.1 in the AACSB accred

In [89]:
data = []

for i, (q, o) in enumerate(zip(test_query, test_out)):
    entry = {"query": q, "output": o, "idx": i}
    data.append(entry)

# Write data to JSON file
with open(f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_advRAG_output.json", "w") as outfile:
    json.dump(data, outfile, indent=4)
